In [1]:
import sys
sys.setrecursionlimit(15000)

import torch
import torch.nn.functional as F
from torch import nn
import numpy as np
from torch.autograd import Variable
from torchvision.datasets.mnist import MNIST
device='cuda:4'
from torchvision import datasets, transforms
import pandas as pd
import numpy as np
from path import Path
import torch.utils.data as data
from imageio import imread
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F 
import matplotlib.pyplot as plt
%matplotlib inline 
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import *

In [2]:
df=pd.read_csv('PROSTRATE_GENE_TO_PDB.csv')
final_df=df[pd.notnull(df['Cross-reference (PDB)'])]
Gene_names=list(final_df['Gene names  (primary )'])
PDB_idz=list(final_df['Cross-reference (PDB)'])
gene_names=[]
for i in range (len(Gene_names)):
    gene_names.append(Gene_names[i].lower())
dictionary = dict(zip(gene_names, PDB_idz))
inp=pd.read_csv('preprocessed_prostrate.csv')
prostrate_gene_names=list(inp['gen_name'])
prostrate_gene_names=[i.lower() for i in prostrate_gene_names]
labels_csv=pd.read_csv('Labels_prostate.txt', sep=" ", header=None)
labels_csv.columns=["a"]
labels=list(labels_csv['a'])
# print(df)
# df.head()
print(len(gene_names))

742


In [3]:
s=0
a=0
b=0
c=0
pdb_to_label={}
for i in prostrate_gene_names:
    
    if i in dictionary.keys():
        pdb_id_=dictionary[i]
        pdbid_list=pdb_id_.split(';')
        for j in pdbid_list: 
            pdb_to_label[j.lower()]=labels[s]  
            if(labels[s]==0):
                a=a+1
            elif labels[s]==1:
                b=b+1
            elif labels[s]==2:
                c=c+1
            else:
                print(labels[s])
        s=s+1
print(a,b,c)

3442 3782 0


In [4]:
# f=0
loc_3D=Path('/home1/pratik/Multi-modality/testrun')
u=loc_3D.dirs()
train_data=[]
pdb_data=[]
label_data=[]
a=0
b=0
c=0
count=0
for q in pdb_to_label.keys():
    exists = os.path.isdir(loc_3D/q)
    if not exists:
        continue
    if q=='':
        continue
    if(pdb_to_label[q]==0):
#         if a==5000:
#             continue
        a=a+count
        count=1
    elif pdb_to_label[q]==1:
#         if b==5000:
#             continue
        b=b+count
        count=1
    elif pdb_to_label[q]==2:
#         if(c==5000):
#             continue
        c=c+count
        count=1

    for u in range(count):
        train_data.append(str(loc_3D/q))
        pdb_data.append(q)
        
print(a,b,c)      

df = pd.DataFrame({'train_pdb':train_data,'pdb_id':pdb_data})
df.to_csv('Prostrate_3D.csv')
print(len(pdb_data))        
    
    

2503 2625 0
5129


In [5]:
def load_as_float(path):
    f = Image.open(path)
    f=f.resize((100,100))
    rgb = f.convert('RGB')
    return(np.array(rgb))
class SequenceFolder(data.Dataset):
    def __init__(self, root):
        self.images_concat,self.all_labels=self.crawl(root)
        
    def crawl(self,csv_loc):
        comp_list=[]
        df=pd.read_csv(csv_loc)
        images_concat=[]
        labels=[]
        for i in range(len(df)):
            print(i+1)
            imp=np.zeros((200,200,3))
            image_loc=Path(df['train_pdb'][i])
            images= sorted((image_loc).files('*.png'))
            if len(images)!=7:
                print("fail-"+str(i))
                continue
            
            try:
                
                imp[0:100,0:100,:]=np.array(load_as_float(images[0]))
                imp[0:100,100:200,:]=np.array(load_as_float(images[1]))
                imp[100:200,0:100,:]=np.array(load_as_float(images[2]))
                imp[100:200,100:200,:]=np.array(load_as_float(images[3]))
                imp
                image_label=pdb_to_label[df['pdb_id'][i]]
            except:
                print("fail-"+str(i))
                continue

            images_concat.append(imp)
            labels.append(image_label)

#             if i==5:
#                 break
        return(images_concat,labels)
        
    def __len__(self):
        return len(self.all_labels)       

    def __getitem__(self, index):
        return np.array(self.images_concat[index]), np.array(self.all_labels[index])
        
        
        
        
        
    
        

In [ ]:
        
train_set=SequenceFolder('Prostrate_3D.csv')     

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [7]:
batch_size = 4
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(train_set)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, 
                                           sampler=train_sampler,drop_last=True)
validation_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                                sampler=valid_sampler,drop_last=True)


In [8]:
# train_loader = torch.utils.data.DataLoader(
#     train_set, batch_size=4, shuffle=True,
#     num_workers=1, pin_memory=False)

In [9]:
len(validation_loader)

256

In [10]:
for a,b in train_loader:
    print(a.shape)

torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4,

torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4,

torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4,

torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4, 200, 200, 3])
torch.Size([4,

In [53]:
BATCH_SIZE = 4
NUM_CLASSES = 2
NUM_EPOCHS = 500
NUM_ROUTING_ITERATIONS = 10

In [54]:
def softmax(input, dim=1):
    transposed_input = input.transpose(dim, len(input.size()) - 1)
    softmaxed_output = F.softmax(transposed_input.contiguous().view(-1, transposed_input.size(-1)), dim=-1)
    return softmaxed_output.view(*transposed_input.size()).transpose(dim, len(input.size()) - 1)

def augmentation(x, max_shift=2):
    _, _, height, width = x.size()

    h_shift, w_shift = np.random.randint(-max_shift, max_shift + 1, size=2)
    source_height_slice = slice(max(0, h_shift), h_shift + height)
    source_width_slice = slice(max(0, w_shift), w_shift + width)
    target_height_slice = slice(max(0, -h_shift), -h_shift + height)
    target_width_slice = slice(max(0, -w_shift), -w_shift + width)

    shifted_image = torch.zeros(*x.size())
    shifted_image[:, :, source_height_slice, source_width_slice] = x[:, :, target_height_slice, target_width_slice]
    return shifted_image.float()

In [55]:
class CapsuleLayer(nn.Module):
    def __init__(self, num_capsules, num_route_nodes, in_channels, out_channels, kernel_size=None, stride=None,
                 num_iterations=NUM_ROUTING_ITERATIONS):
        super(CapsuleLayer, self).__init__()

        self.num_route_nodes = num_route_nodes
        self.num_iterations = num_iterations

        self.num_capsules = num_capsules

        if num_route_nodes != -1:
            self.route_weights = nn.Parameter(torch.randn(num_capsules, num_route_nodes, in_channels, out_channels))
        else:
            self.capsules = nn.ModuleList(
                [nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=0) for _ in
                 range(num_capsules)])

    def squash(self, tensor, dim=-1):
        squared_norm = (tensor ** 2).sum(dim=dim, keepdim=True)
        scale = squared_norm / (1 + squared_norm)
        return scale * tensor / torch.sqrt(squared_norm)

    def forward(self, x):
        if self.num_route_nodes != -1:
#             print("x",x[None, :, :, None, :].shape)
#             print("route",self.route_weights[:, None, :, :, :].shape)
            priors = x[None, :, :, None, :] @ self.route_weights[:, None, :, :, :]
#             print(priors.shape)

            logits = Variable(torch.zeros(*priors.size())).to(device)
#             print("logits-"+str(logits.shape))
            for i in range(self.num_iterations):
                probs = softmax(logits, dim=2)
                
                outputs = self.squash((probs * priors).sum(dim=2, keepdim=True))
#                 print("onii",outputs.shape)

                if i != self.num_iterations - 1:
                    delta_logits = (priors * outputs).sum(dim=-1, keepdim=True)
#                     print("war",delta_logits.shape)
                    logits = logits + delta_logits
        else:
            outputs = [capsule(x).view(x.size(0), -1, 1) for capsule in self.capsules]
#             print("outputs-"+str(outputs[0].shape))
            outputs = torch.cat(outputs, dim=-1)
            outputs = self.squash(outputs)

        return outputs

In [56]:
class CapsuleNet(nn.Module):
    def __init__(self):
        super(CapsuleNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=16, stride=1)
        self.pool=nn.MaxPool2d(3)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=12, stride=1)
        
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=9, stride=1)
        self.relu=nn.ReLU()
#         self.dummy_linear=nn.Linear(199712,32*16*16)
        self.primary_capsules = CapsuleLayer(num_capsules=8, num_route_nodes=-1, in_channels=256, out_channels=32,
                                             kernel_size=9, stride=2)
        self.digit_capsules = CapsuleLayer(num_capsules=NUM_CLASSES, num_route_nodes=9248, in_channels=8,
                                           out_channels=16)

        self.decoder = nn.Sequential(
            nn.Linear(16 * NUM_CLASSES, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 784),
            nn.Sigmoid()
        )

    def forward(self, x, y=None):
        x =F.relu(self.conv3((F.relu(self.conv2(self.pool(F.relu(self.conv1(x), inplace=True)))))))
#         print(x.shape)
        x = self.primary_capsules(x)
#         print(x.shape)
        
#         dummy=x.permute(0,2,1)
#         dummy=dummy_linear(dummy)
#         x=dummy.permute(0,2,1)
        x = self.digit_capsules(x).squeeze().transpose(0, 1)
         

        classes = (x ** 2).sum(dim=-1) ** 0.5
        classes = F.softmax(classes, dim=-1)

        if y is None:
            # In all batches, get the most active capsule.
            #print(classes.shape)
            _, max_length_indices = classes.max(dim=1)
            y = Variable(torch.eye(NUM_CLASSES)).to(device).index_select(dim=0, index=max_length_indices.data)

        reconstructions = self.decoder((x * y[:, :, None]).view(x.size(0), -1))
#         print(classes.shape)
        return classes, reconstructions




In [57]:
# u=CapsuleNet().to(device)
# z=u(torch.zeros(4,3,200,200).to(device))
# print(z[0].shape)
# # print(z[1].shape)

In [58]:
# z=CapsuleLoss()
# zx=z(torch.zeros(7,1,28,28),torch.rand(7,10),torch.rand(7,10),torch.zeros(7,28*28))

In [59]:
class CapsuleLoss(nn.Module):
    def __init__(self):
        super(CapsuleLoss, self).__init__()
        self.reconstruction_loss = nn.MSELoss(size_average=False)

    def forward(self, images, labels, classes, reconstructions):
        left = F.relu(0.9 - classes, inplace=True) ** 2
        right = F.relu(classes - 0.1, inplace=True) ** 2
#         print(left.shape)
#         print(labels.shape)
        margin_loss = labels * left + 0.5 * (1. - labels) * right
        margin_loss = margin_loss.sum()


        reconstruction_loss = 0#self.reconstruction_loss(reconstructions, images)

        return (margin_loss + 0.0005 * reconstruction_loss) / images.size(0)

In [60]:
print(len(train_loader))
# for batch_idx, (data, target) in enumerate(train_loader):
#         print(target.shape)

1025


In [61]:
model=CapsuleNet().to(device)
criterion=CapsuleLoss()
optim_params = [
    {'params': model.parameters(), 'lr': 0.0001}
]
optimizer = torch.optim.Adam(optim_params)


In [62]:
epoch=1000
for i in range(epoch):
    loss_epoch=0
    accuracy=0
    total_correct=0
    total=0
    u=0
    confusion_matrix = torch.zeros(2, 2)
    
    for data,target in (validation_loader):

#          print(data.shape)
#         print(target.shape)
        data=data.permute(0,3,1,2).float()
#         print(torch.mean(data))
#         print(data.shape)

        classes, reconstructions=model(data.to(device))
        
        x,y=torch.max(classes,1)
#         print(y.shape)
#         print(target.shape)
#         print(y)
#         print(target)
        _, preds = torch.max(classes, 1)
        total_correct=total_correct+torch.sum(y.long()==target.long().to(device))
        total=total+data.shape[0]
        for t, p in zip(target.view(-1), preds.view(-1)):
#             print(t,p)
#             print(t,p)
            confusion_matrix[t.long(), p.long()] += 1
    print(total_correct,total)
    print(confusion_matrix)
    print("saving")
    torch.save(model.state_dict(), 'prostrate_capsulenet3d_ankit.pt')
    print("saving finished")
    for data,target in  (train_loader):

        
        
        
        data=data.permute(0,3,1,2).float()
        data=data.to(device)
        target
        classes, reconstructions=model(data.to(device))
        
        
#         print(x.shape)
        optimizer.zero_grad()
        label= torch.eye(2).index_select(dim=0, index=target)
        label=label.to(device)
#         print(data.shape)
#         print(label.shape)
#         print(classes.shape)
#         print(reconstructions.shape)
        loss=criterion(data,label,classes,reconstructions)
        loss_epoch=loss_epoch+loss
        loss.backward()
        optimizer.step()
#         print(u)
#         u=u+1
        

    print(loss_epoch)
     
    
          

tensor(506, device='cuda:4') 1024
tensor([[391., 123.],
        [395., 115.]])
saving
saving finished
tensor(245.9475, device='cuda:4', grad_fn=<AddBackward0>)
tensor(526, device='cuda:4') 1024
tensor([[ 27., 487.],
        [ 11., 499.]])
saving
saving finished
tensor(241.0697, device='cuda:4', grad_fn=<AddBackward0>)
tensor(564, device='cuda:4') 1024
tensor([[ 81., 433.],
        [ 27., 483.]])
saving
saving finished
tensor(228.3715, device='cuda:4', grad_fn=<AddBackward0>)
tensor(560, device='cuda:4') 1024
tensor([[159., 355.],
        [109., 401.]])
saving
saving finished
tensor(212.0509, device='cuda:4', grad_fn=<AddBackward0>)
tensor(648, device='cuda:4') 1024
tensor([[272., 243.],
        [133., 376.]])
saving
saving finished
tensor(177.5221, device='cuda:4', grad_fn=<AddBackward0>)
tensor(688, device='cuda:4') 1024
tensor([[297., 217.],
        [119., 391.]])
saving
saving finished
tensor(138.8306, device='cuda:4', grad_fn=<AddBackward0>)
tensor(707, device='cuda:4') 1024
tensor

tensor(45.8621, device='cuda:4', grad_fn=<AddBackward0>)
tensor(732, device='cuda:4') 1024
tensor([[346., 168.],
        [124., 386.]])
saving
saving finished
tensor(46.5925, device='cuda:4', grad_fn=<AddBackward0>)
tensor(734, device='cuda:4') 1024
tensor([[354., 160.],
        [130., 380.]])
saving
saving finished
tensor(45.6870, device='cuda:4', grad_fn=<AddBackward0>)
tensor(739, device='cuda:4') 1024
tensor([[369., 146.],
        [139., 370.]])
saving
saving finished
tensor(45.2225, device='cuda:4', grad_fn=<AddBackward0>)
tensor(743, device='cuda:4') 1024
tensor([[361., 153.],
        [128., 382.]])
saving
saving finished
tensor(44.9944, device='cuda:4', grad_fn=<AddBackward0>)
tensor(737, device='cuda:4') 1024
tensor([[343., 172.],
        [115., 394.]])
saving
saving finished
tensor(46.4624, device='cuda:4', grad_fn=<AddBackward0>)
tensor(746, device='cuda:4') 1024
tensor([[358., 157.],
        [121., 388.]])
saving
saving finished
tensor(45.2713, device='cuda:4', grad_fn=<AddB

saving finished
tensor(44.6080, device='cuda:4', grad_fn=<AddBackward0>)
tensor(747, device='cuda:4') 1024
tensor([[352., 163.],
        [114., 395.]])
saving
saving finished
tensor(45.6178, device='cuda:4', grad_fn=<AddBackward0>)
tensor(738, device='cuda:4') 1024
tensor([[354., 160.],
        [126., 384.]])
saving
saving finished
tensor(45.0051, device='cuda:4', grad_fn=<AddBackward0>)
tensor(738, device='cuda:4') 1024
tensor([[344., 171.],
        [115., 394.]])
saving
saving finished
tensor(44.8087, device='cuda:4', grad_fn=<AddBackward0>)
tensor(732, device='cuda:4') 1024
tensor([[350., 164.],
        [128., 382.]])
saving
saving finished
tensor(44.7080, device='cuda:4', grad_fn=<AddBackward0>)
tensor(735, device='cuda:4') 1024
tensor([[355., 160.],
        [129., 380.]])
saving
saving finished
tensor(44.6006, device='cuda:4', grad_fn=<AddBackward0>)
tensor(734, device='cuda:4') 1024
tensor([[356., 158.],
        [132., 378.]])
saving
saving finished
tensor(44.5903, device='cuda:4

saving finished
tensor(44.7432, device='cuda:4', grad_fn=<AddBackward0>)
tensor(728, device='cuda:4') 1024
tensor([[398., 117.],
        [179., 330.]])
saving
saving finished
tensor(45.4362, device='cuda:4', grad_fn=<AddBackward0>)
tensor(721, device='cuda:4') 1024
tensor([[348., 166.],
        [137., 373.]])
saving
saving finished
tensor(45.0176, device='cuda:4', grad_fn=<AddBackward0>)
tensor(739, device='cuda:4') 1024
tensor([[375., 139.],
        [146., 364.]])
saving
saving finished
tensor(45.0204, device='cuda:4', grad_fn=<AddBackward0>)
tensor(748, device='cuda:4') 1024
tensor([[359., 155.],
        [121., 389.]])
saving
saving finished
tensor(44.6882, device='cuda:4', grad_fn=<AddBackward0>)
tensor(752, device='cuda:4') 1024
tensor([[367., 147.],
        [125., 385.]])
saving
saving finished
tensor(44.5778, device='cuda:4', grad_fn=<AddBackward0>)
tensor(752, device='cuda:4') 1024
tensor([[364., 151.],
        [121., 388.]])
saving
saving finished
tensor(44.5325, device='cuda:4

tensor(44.8648, device='cuda:4', grad_fn=<AddBackward0>)
tensor(744, device='cuda:4') 1024
tensor([[337., 177.],
        [103., 407.]])
saving
saving finished
tensor(45.0491, device='cuda:4', grad_fn=<AddBackward0>)
tensor(738, device='cuda:4') 1024
tensor([[347., 167.],
        [119., 391.]])
saving
saving finished
tensor(44.6239, device='cuda:4', grad_fn=<AddBackward0>)
tensor(735, device='cuda:4') 1024
tensor([[368., 146.],
        [143., 367.]])
saving
saving finished
tensor(44.8020, device='cuda:4', grad_fn=<AddBackward0>)
tensor(739, device='cuda:4') 1024
tensor([[347., 167.],
        [118., 392.]])
saving
saving finished
tensor(44.6369, device='cuda:4', grad_fn=<AddBackward0>)
tensor(744, device='cuda:4') 1024
tensor([[362., 152.],
        [128., 382.]])
saving
saving finished
tensor(44.4718, device='cuda:4', grad_fn=<AddBackward0>)
tensor(749, device='cuda:4') 1024
tensor([[355., 159.],
        [116., 394.]])
saving
saving finished
tensor(44.3956, device='cuda:4', grad_fn=<AddB

...

saving finished
tensor(44.3060, device='cuda:4', grad_fn=<AddBackward0>)
tensor(747, device='cuda:4') 1024
tensor([[348., 166.],
        [111., 399.]])
saving
saving finished
tensor(44.2498, device='cuda:4', grad_fn=<AddBackward0>)
tensor(752, device='cuda:4') 1024
tensor([[364., 150.],
        [122., 388.]])
saving
saving finished
tensor(44.2614, device='cuda:4', grad_fn=<AddBackward0>)
tensor(739, device='cuda:4') 1024
tensor([[324., 190.],
        [ 95., 415.]])
saving
saving finished
tensor(44.6213, device='cuda:4', grad_fn=<AddBackward0>)
tensor(752, device='cuda:4') 1024
tensor([[349., 165.],
        [107., 403.]])
saving
saving finished
tensor(44.4375, device='cuda:4', grad_fn=<AddBackward0>)
tensor(752, device='cuda:4') 1024
tensor([[367., 147.],
        [125., 385.]])
saving
saving finished
tensor(44.4646, device='cuda:4', grad_fn=<AddBackward0>)
tensor(753, device='cuda:4') 1024
tensor([[356., 158.],
        [113., 397.]])
saving
saving finished
tensor(44.4312, device='cuda:4

tensor(44.0770, device='cuda:4', grad_fn=<AddBackward0>)
tensor(744, device='cuda:4') 1024
tensor([[354., 161.],
        [119., 390.]])
saving
saving finished
tensor(44.0755, device='cuda:4', grad_fn=<AddBackward0>)
tensor(744, device='cuda:4') 1024
tensor([[356., 159.],
        [121., 388.]])
saving
saving finished
tensor(44.0741, device='cuda:4', grad_fn=<AddBackward0>)
tensor(745, device='cuda:4') 1024
tensor([[356., 159.],
        [120., 389.]])
saving
saving finished
tensor(44.0729, device='cuda:4', grad_fn=<AddBackward0>)
tensor(744, device='cuda:4') 1024
tensor([[353., 161.],
        [119., 391.]])
saving
saving finished
tensor(44.0718, device='cuda:4', grad_fn=<AddBackward0>)
tensor(743, device='cuda:4') 1024
tensor([[351., 163.],
        [118., 392.]])
saving
saving finished
tensor(44.0707, device='cuda:4', grad_fn=<AddBackward0>)
tensor(745, device='cuda:4') 1024
tensor([[354., 160.],
        [119., 391.]])
saving
saving finished
tensor(44.0698, device='cuda:4', grad_fn=<AddB

tensor(44.4535, device='cuda:4', grad_fn=<AddBackward0>)
tensor(737, device='cuda:4') 1024
tensor([[354., 160.],
        [127., 383.]])
saving
saving finished
tensor(44.4343, device='cuda:4', grad_fn=<AddBackward0>)
tensor(734, device='cuda:4') 1024
tensor([[354., 160.],
        [130., 380.]])
saving
saving finished
tensor(44.4245, device='cuda:4', grad_fn=<AddBackward0>)
tensor(742, device='cuda:4') 1024
tensor([[361., 153.],
        [129., 381.]])
saving
saving finished
tensor(44.7222, device='cuda:4', grad_fn=<AddBackward0>)
tensor(733, device='cuda:4') 1024
tensor([[334., 180.],
        [111., 399.]])
saving
saving finished
tensor(44.6436, device='cuda:4', grad_fn=<AddBackward0>)
tensor(732, device='cuda:4') 1024
tensor([[329., 186.],
        [106., 403.]])
saving
saving finished
tensor(44.4609, device='cuda:4', grad_fn=<AddBackward0>)
tensor(733, device='cuda:4') 1024
tensor([[328., 186.],
        [105., 405.]])
saving
saving finished
tensor(44.1937, device='cuda:4', grad_fn=<AddB

saving finished
tensor(44.3096, device='cuda:4', grad_fn=<AddBackward0>)
tensor(732, device='cuda:4') 1024
tensor([[331., 183.],
        [109., 401.]])
saving
saving finished
tensor(45.2825, device='cuda:4', grad_fn=<AddBackward0>)
tensor(737, device='cuda:4') 1024
tensor([[333., 181.],
        [106., 404.]])
saving
saving finished
tensor(44.6091, device='cuda:4', grad_fn=<AddBackward0>)
tensor(739, device='cuda:4') 1024
tensor([[354., 161.],
        [124., 385.]])
saving
saving finished
tensor(44.3865, device='cuda:4', grad_fn=<AddBackward0>)
tensor(740, device='cuda:4') 1024
tensor([[347., 167.],
        [117., 393.]])
saving
saving finished
tensor(44.3726, device='cuda:4', grad_fn=<AddBackward0>)
tensor(735, device='cuda:4') 1024
tensor([[363., 151.],
        [138., 372.]])
saving
saving finished
tensor(44.2603, device='cuda:4', grad_fn=<AddBackward0>)
tensor(738, device='cuda:4') 1024
tensor([[358., 156.],
        [130., 380.]])
saving
saving finished
tensor(44.2051, device='cuda:4

KeyboardInterrupt: 

In [63]:
torch.save(model.state_dict(), 'percentcapsulenet3d_good.pt')

In [ ]:
#3459